## Общая схема кросс валидации

1. Из датафрейма берется та часть пользователей, которая кликала от 1 до 5 мероприятий. Они принадлежат 95 % всей выборки
2. Из этой части случайным образом выбирается один пользователь с фиксированным числом кликнутых мероприятий от 3 до 5
3. Получаем список кликнутых пользователем мероприятий
4. Цикл по кликнутым мероприятиям:
4.1 Из общей выборки исключается строка с записью о клике выбранного пользователя на мероприятие
4.2 Обучение модели по данным с исключенным мероприятием
4.3 Получение списка из N мероприятий для данного пользователя
4.4 Проверяем, есть ли исключенное мероприятие в списке рекомендованных

In [1]:
import pandas as pd
import numpy as np
from preprocessing import data_management as dm
from sklearn.model_selection import ParameterGrid
from recommenders.implicit_models import UserItemRecommender, ALSRecommender

In [2]:
def split_df_into_diapasons(df: pd.DataFrame) -> pd.DataFrame:
    activity = df.groupby('user_id')['clicks_count'].count().sort_values(ascending=False).reset_index()
    activity['diapason'] = pd.cut(activity['clicks_count'], bins=np.linspace(0, 70, 15),
                                  labels=[f'({i}:{j}]' for i, j in zip(range(0, 70, 5), range(5, 75, 5))])
    return activity

In [8]:
def evaluate(
        user_id: str,
        user_event_df: pd.DataFrame,
        recommender: UserItemRecommender,
        number_of_recommended=10
) -> float:
    clicked_by_user = user_event_df.loc[user_event_df.user_id == user_id]

    count = 0
    for _, clicked_event in clicked_by_user.iterrows():
        example = user_event_df[(user_event_df.user_id == clicked_event.user_id) &
                                (user_event_df.item_id == clicked_event.item_id)]
        us_ev = user_event_df.drop(example.index)
        recommender.fit(user_item_df=us_ev)
        recommended_to_user = recommender.get_user_recommendation(user_id, number_of_recommended, as_pd_dataframe=True)

        if example.iloc[0].item_id in list(recommended_to_user.item_id.values):
            count += 1

    return round(count / len(clicked_by_user), 2)

In [10]:
def cross_validation(
        user_activity_df: pd.DataFrame,
        user_event_df: pd.DataFrame,
        num_of_clicked: int = 5,
        num_of_recommended: int = 10
):
    users_in_interval = user_activity_df.loc[
        (user_activity_df.diapason == '(0:5]') &
        (user_activity_df.clicks_count == num_of_clicked)
        ]

    als_parameters = {
        'factors': [i for i in range(20, 101)],
        'regularization': [j for j in np.linspace(0, 1, 11, endpoint=False)],
        'iterations': [k for k in range(100, 200, 20)],
        'confidence': ['alpha'],
        'alpha_value': [g for g in range(5, 21)]
    }
    user_from_interval = users_in_interval.sample().iloc[0]
    best_params = dict()
    max_probability = 0
    for params in ParameterGrid(als_parameters):
        recommender = ALSRecommender(**params)
        probability = evaluate(
            user_id=user_from_interval.user_id,
            user_event_df=user_event_df,
            recommender=recommender,
            number_of_recommended=num_of_recommended
        )
        if probability > max_probability:
            max_probability = probability
            best_params = params
        break

    print(max_probability, best_params)


In [5]:
bashkortostan_user_event = pd.read_csv('../dataset/user_event_dfs/bashkortostan_user_event_df.csv')
bashkortostan_user_event.drop(columns='Unnamed: 0', inplace=True)
bashkortostan_activity = split_df_into_diapasons(df=bashkortostan_user_event)
bashkortostan_activity

,user_id,clicks_count,diapason
0,e4f8e204a900fa2b4ad6f3fbb48b5585,36,(35:40]
1,bdfae6b594bc1f28467b4f2db002746e,35,(30:35]
2,df35377e953a13a7ad537ba04737b826,30,(25:30]
3,4e3f977e66736abb763b3fcab57b122b,28,(25:30]
4,2b0c5f923468dc1b1d199ea46f503022,28,(25:30]
...,...,...,...
5870,33ca62f0ef71ecf58e3ae098c2957388,1,(0:5]
5871,33bf851266570c7d0572d531da46bae9,1,(0:5]
5872,8c90eb1e907a4799995f8cd255e2d488,1,(0:5]
5873,8cb19200880fb3ab173feb9df18b4c5e,1,(0:5]


In [6]:
bashkortostan_user_event = dm.numerate_user_event_df(bashkortostan_user_event)
bashkortostan_user_event = bashkortostan_user_event.rename(columns={
    'event_id': 'item_id',
    'event_num': 'item_num',
    'clicks_count': 'rating'
})

In [11]:
cross_validation(
    user_event_df=bashkortostan_user_event,
    user_activity_df=bashkortostan_activity,
    num_of_clicked=5,
    num_of_recommended=10
)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

0 {}
